In [1]:
n = 10^3

in_ = collect(1:n)

fl_ = float.(in_)

ch_ = [Char(i[1]) for i in in_]

st_ = string.(in_);

;

`Int`, the smallest concrete type here, is the fastest: 

`String` is not concrete only after one is created and the computer knows its size, so it is slow to work with:

In [2]:
for ty in (Int, Float64, Char, String)
    
    print("$ty: ")
    
    try
        
        println(sizeof(ty))
        
    catch er
        
        println(er.msg)
        
    end
    
end 

Int64: 8
Float64: 8
Char: 4
String: Type String does not have a definite size.


`unique` allocates the most, so it is the slowest (avoid it):

In [3]:
@btime unique($in_)

@btime unique($fl_)

@btime unique($ch_)

@btime unique($st_)

;

  196.645 ns (7 allocations: 880 bytes)
  226.832 ns (7 allocations: 880 bytes)
  197.196 ns (7 allocations: 624 bytes)
  401.328 ns (7 allocations: 848 bytes)


`Set` always allocates 7 times and is faster:

In [4]:
@btime Set($in_)

@btime Set($fl_)

@btime Set($ch_)

@btime Set($st_)

;

  98.865 ns (4 allocations: 400 bytes)
  127.690 ns (4 allocations: 400 bytes)
  102.533 ns (4 allocations: 336 bytes)
  197.285 ns (4 allocations: 384 bytes)


`unique!` allocates 0 for the number types and more than 7 for the letter types.

So it is the fastest for working with the numbers and slower than `Set` for the letters:

In [5]:
@btime unique!($in_)

@btime unique!($fl_)

@btime unique!($ch_)

@btime unique!($st_)

;

  18.202 ns (0 allocations: 0 bytes)
  24.389 ns (0 allocations: 0 bytes)
  123.846 ns (4 allocations: 336 bytes)
  350.467 ns (4 allocations: 384 bytes)


`Set` is the fastest for working with `Char` and `String`.

`unique!` is the fastest for working with `Int` and `Float64`.

However, the content matters.

Contents with repeats slow down `Set` more:

In [10]:
inr_ = repeat(1:10, n)

@btime unique($inr_)

@btime Set($inr_)

@btime unique!($inr_)

;

  21.792 μs (7 allocations: 880 bytes)
  29.791 μs (7 allocations: 144.59 KiB)
  18.202 ns (0 allocations: 0 bytes)
